<h1>Validation of new contexts before updating the original embeddings so that the output quality of the LLM does not degrade due to hallucinations caused by irrelevant contexts.</h1>

<h2 style="color:red">Importing libraries, creating the original vector store and embeddings.</h2>

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.document_loaders import PyPDFLoader, WebBaseLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from uuid import uuid4
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_core.documents import Document

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector_store = Chroma(
    collection_name="aerospace",
    embedding_function=embeddings,
    persist_directory="./aerospace_db",  
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-10-13 07:54:32.825649: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
docs = [
    Document(
        page_content="The Boeing Company is an American multinational corporation that designs, manufactures, and sells airplanes, rotorcraft, rockets, satellites, and missiles worldwide.The company also provides leasing and product support services. Boeing is among the largest global aerospace manufacturers; it is the fourth-largest defense contractor in the world based on 2022 revenue and is the largest exporter in the United States by dollar value. Boeing was founded by William Boeing in Seattle, Washington, on July 15, 1916.The present corporation is the result of the merger of Boeing with McDonnell Douglas on August 1, 1997. As of 2023, the Boeing Company's corporate headquarters is located in the Crystal City neighborhood of Arlington County, Virginia.The company is organized into three primary divisions: Boeing Commercial Airplanes (BCA), Boeing Defense, Space & Security (BDS) and Boeing Global Services (BGS). In 2021, Boeing recorded $62.3 billion in sales. Boeing is ranked 54th on the Fortune 500 list (2020), and ranked 121st on the Fortune Global 500 list (2020). On October 12, 2024 the company announced its intention to cut 17,000 jobs, about 10% of its global workforce."
    ),
    Document(
        page_content="Airbus SE is a European[8] aerospace corporation. The company's primary business is the design and manufacturing of commercial aircraft but it also has separate defence and space and helicopter divisions. Since 2019, Airbus has been the world's largest manufacturer of airliners as well as the leading helicopter manufacturer.The company was incorporated as the European Aeronautic Defence and Space Company (EADS) in the year 2000 through the merger of the French Aérospatiale-Matra, the German DASA and Spanish CASA. The new entity subsequently acquired full ownership of its subsidiary, Airbus Industrie GIE, a joint venture of European aerospace companies originally incorporated in 1970 to develop and produce a wide-body aircraft to compete with American-built airliners. EADS rebranded itself as Airbus SE in 2015. Reflecting its multinational origin, the company operates major offices and assembly plants in France, Germany, Spain, and the United Kingdom, along with more recent additions in Canada, China, and the United States.Airbus' headquarters are legally registered in Leiden, Netherlands, but daily management is conducted from the company's main office located in Blagnac, France. The SE in its corporate name stands for Societas Europaea.The company is led by CEO Guillaume Faury and is a component of the EURO STOXX 50 stock market index.Since its inception in 2000, the company's shares have been listed on the Paris Stock Exchange, the Frankfurt Stock Exchange and the four regional Spanish stock exchanges (including the Bolsa de Madrid).In 2024, following similar ventures launched in Singapore and the Netherlands, Airbus announced its plans to establish a Tech Hub in Japan to foster partnerships and prepare for the next generation of aircraft."
    ),
    Document(
        page_content="Pratt & Whitney is an American aerospace manufacturer with global service operations.[2] It is a subsidiary of RTX Corporation (formerly Raytheon Technologies). Pratt & Whitney's aircraft engines are widely used in both civil aviation (especially airlines) and military aviation. Its headquarters are in East Hartford, Connecticut.[3] The company is the world's second largest commercial aircraft engine manufacturer, with a 35% market share as of 2020.[4] In addition to aircraft engines, Pratt & Whitney manufactures gas turbine engines for industrial use, marine propulsion, and power generation. In 2017, the company reported that it supported more than 11,000 customers in 180 countries around the world."
    )
]

uuids = [str(uuid4()) for _ in range(len(docs))]

In [4]:
vector_store.add_documents(documents=docs, ids=uuids)

['7ac251b4-aab9-4888-89d9-f91fe0f828d0',
 'c53aa123-4f0a-4c41-ab5d-bc177976e26f',
 'd8e272b0-0ddb-4565-9004-b0c38abe8403']

In [5]:
results = vector_store.similarity_search_with_score(
    "What are the documents about?", k=1
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=1.758190] The Boeing Company is an American multinational corporation that designs, manufactures, and sells airplanes, rotorcraft, rockets, satellites, and missiles worldwide.The company also provides leasing and product support services. Boeing is among the largest global aerospace manufacturers; it is the fourth-largest defense contractor in the world based on 2022 revenue and is the largest exporter in the United States by dollar value. Boeing was founded by William Boeing in Seattle, Washington, on July 15, 1916.The present corporation is the result of the merger of Boeing with McDonnell Douglas on August 1, 1997. As of 2023, the Boeing Company's corporate headquarters is located in the Crystal City neighborhood of Arlington County, Virginia.The company is organized into three primary divisions: Boeing Commercial Airplanes (BCA), Boeing Defense, Space & Security (BDS) and Boeing Global Services (BGS). In 2021, Boeing recorded $62.3 billion in sales. Boeing is ranked 54th on 

In [8]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

local_llm = "gemma2"
llm = ChatOllama(model=local_llm, keep_alive="3h", max_tokens=512, temperature=0)

template = """Answer the question based only on the following context and extract out a meaningful answeer. 
If the context doesn't contain an answer, just respond that you are unable to find an answer.

CONTEXT: {context}

QUESTION: {question}

ANSWER: 
"""

prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt | llm
)

def ask_question(question):
    print("Answer: \n\n", end = "", flush = True)
    for chunk in rag_chain.stream(question):
        print(chunk.content, end="", flush=True)
    print("\n")

In [27]:
ask_question("What is Boeing?")
ask_question("What kind of clothes does american eagle sell?")

Answer: 



Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


Boeing is an American multinational corporation that designs, manufactures, and sells airplanes, rotorcraft, rockets, satellites, and missiles worldwide.  The company also provides leasing and product support services. 


Answer: 



Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


I am unable to find an answer.  




<h2 style="color: red">Creating a new vector store with a completely different topic and asking the LLM to compare between the new and old embeddings.</h2>

1. Amphibians (Irrelevant)
2. Kennedy Space Center (Somewhat related but still not relevant to the original contexts)
3. Rolls Royce Trent (Very relevant)

In [11]:
new_docs = [
    Document(
        page_content="Amphibians are ectothermic, anamniotic, four-limbed vertebrate animals that constitute the class Amphibia. In its broadest sense, it is a paraphyletic group encompassing all tetrapods excluding the amniotes (tetrapods with an amniotic membrane, such as modern reptiles, birds and mammals). All extant (living) amphibians belong to the monophyletic subclass Lissamphibia, with three living orders: Anura (frogs and toads), Urodela (salamanders), and Gymnophiona (caecilians). Evolved to be mostly semiaquatic, amphibians have adapted to inhabit a wide variety of habitats, with most species living in freshwater, wetland or terrestrial ecosystems (such as riparian woodland, fossorial and even arboreal habitats). Their life cycle typically starts out as aquatic larvae with gills known as tadpoles, but some species have developed behavioural adaptations to bypass this."
    )
]

new_uuids = [str(uuid4()) for _ in range(len(new_docs))]

tmp_vector_store = Chroma(
    collection_name="tmp",
    embedding_function=embeddings,
    persist_directory="./tmp_db",  
)

tmp_vector_store.add_documents(documents=new_docs, ids=new_uuids)
new_retriever = tmp_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [20]:
comparison_question = """
    I need you to compare two contexts that are given. The original_context will be the benchmark against which the new_context must be 
    compared.

    After comparison, retrieve a similarlity score between 0-100 where 0 indicates no similarity and 100 indicates highest similarity.
    The score must be based on how related both contexts are and not the exact words. You must ignore any English stop words such as 'The', 
    'and', 'a', 'an' etc. 

    If the score is below 70, I need 3 reasons for the low score listed in bullet points. 

    original_context = {original_context}
    new_context = {new_context}

    question={question}
    ANSWER: 
"""

comparison_prompt = ChatPromptTemplate.from_template(comparison_question)
comparison_rag_chain = ({"original_context": retriever, "new_context": new_retriever, "question": RunnablePassthrough()} | comparison_prompt | llm)

In [23]:
def compare_contexts():
    print("Answer: \n\n", end = "", flush = True)
    question="are both contexts talking about a similar topic?"
    for chunk in comparison_rag_chain.stream(question):
        print(chunk.content, end="", flush=True)
    print("\n")

In [24]:
compare_contexts()

Answer: 



Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


The similarity score between the original_context and new_context is **10**.  

Here's why:

* **Completely Different Topics:** The original context focuses on aerospace corporations like Airbus, Boeing, and Pratt & Whitney. The new context discusses amphibians, their biology, and life cycles. 
* **No Overlapping Concepts:** There are no shared concepts or themes between the two contexts. Aerospace manufacturing and amphibian biology are entirely distinct fields.
* **Different Domains of Knowledge:** The information presented in each context belongs to different domains of knowledge – technology/industry versus biology/zoology.


Let me know if you'd like to compare other contexts! 




In [25]:
new_docs_2 = [
    Document(
        page_content="The John F. Kennedy Space Center (KSC, originally known as the NASA Launch Operations Center), located on Merritt Island, Florida, is one of the National Aeronautics and Space Administration's (NASA) ten field centers. Since 1968, KSC has been NASA's primary launch center of American spaceflight, research, and technology. Launch operations for the Apollo, Skylab and Space Shuttle programs were carried out from Kennedy Space Center Launch Complex 39 and managed by KSC.[4] Located on the east coast of Florida, KSC is adjacent to Cape Canaveral Space Force Station (CCSFS). The management of the two entities work very closely together, share resources, and operate facilities on each other's property.Though the first Apollo flights and all Project Mercury and Project Gemini flights took off from the then-Cape Canaveral Air Force Station, the launches were managed by KSC and its previous organization, the Launch Operations Directorate.[5][6] Starting with the fourth Gemini mission, the NASA launch control center in Florida (Mercury Control Center, later the Launch Control Center) began handing off control of the vehicle to the Mission Control Center in Houston, shortly after liftoff; in prior missions it held control throughout the entire mission."
    )
]

new_uuids_2 = [str(uuid4()) for _ in range(len(new_docs_2))]

tmp_vector_store_2 = Chroma(
    collection_name="tmp_2",
    embedding_function=embeddings,
    persist_directory="./tmp_2_db",  
)

tmp_vector_store_2.add_documents(documents=new_docs_2, ids=new_uuids_2)
new_retriever_2 = tmp_vector_store_2.as_retriever(search_type="similarity", search_kwargs={"k": 3})
comparison_rag_chain_2 = ({"original_context": retriever, "new_context": new_retriever_2, "question": RunnablePassthrough()} | comparison_prompt | llm)


def compare_contexts_2():
    print("Answer: \n\n", end = "", flush = True)
    question="are both contexts talking about a similar topic?"
    for chunk in comparison_rag_chain_2.stream(question):
        print(chunk.content, end="", flush=True)
    print("\n")

compare_contexts_2()

Answer: 



Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


The similarity score between the original_context and new_context is **45**.  

Here are three reasons for the low score:

* **Different Industries:** The original context focuses on aerospace manufacturing companies (Airbus, Boeing, Pratt & Whitney), while the new context discusses a space launch center (John F. Kennedy Space Center). These are related industries but have distinct areas of focus.
* **Lack of Overlapping Entities:** There are no common companies or individuals mentioned in both contexts. 
* **Different Scope:** The original context provides information about the history, operations, and market position of aerospace manufacturers. The new context focuses on a specific space launch center's role in American spaceflight history and its relationship with other entities. 


Let me know if you have any other contexts you'd like me to compare! 




In [26]:
new_docs_3 = [
    Document(
        page_content="The Rolls-Royce Trent is a family of high-bypass turbofans produced by Rolls-Royce. It continues the three spool architecture of the RB211 with a maximum thrust ranging from 61,900 to 97,000 lbf (275 to 431 kN). Launched as the RB-211-524L in June 1988, the prototype first ran in August 1990. Its first variant is the Trent 700 introduced on the Airbus A330 in March 1995, then the Trent 800 for the Boeing 777 (1996), the Trent 500 for the A340 (2002), the Trent 900 for the A380 (2007), the Trent 1000 for the Boeing 787 (2011), the Trent XWB for the A350 (2015), and the Trent 7000 for the A330neo (2018). It has also marine and industrial variants like the RR MT30."
    )
]

new_uuids_3 = [str(uuid4()) for _ in range(len(new_docs_3))]

tmp_vector_store_3 = Chroma(
    collection_name="tmp_3",
    embedding_function=embeddings,
    persist_directory="./tmp_3_db",  
)

tmp_vector_store_3.add_documents(documents=new_docs_3, ids=new_uuids_3)
new_retriever_3 = tmp_vector_store_3.as_retriever(search_type="similarity", search_kwargs={"k": 3})
comparison_rag_chain_3 = ({"original_context": retriever, "new_context": new_retriever_3, "question": RunnablePassthrough()} | comparison_prompt | llm)


def compare_contexts_3():
    print("Answer: \n\n", end = "", flush = True)
    question="are both contexts talking about a similar topic?"
    for chunk in comparison_rag_chain_3.stream(question):
        print(chunk.content, end="", flush=True)
    print("\n")

compare_contexts_3()

Answer: 



Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


## Similarity Score: 85

**Reasoning:** Both contexts discuss prominent aerospace companies and their products.  The original context focuses on major aircraft manufacturers like Airbus, Boeing, while the new context delves into Rolls-Royce's Trent engine family, a crucial component in many commercial aircraft.  

While not directly comparing companies, both contexts highlight key players and innovations within the aerospace industry.


Let me know if you'd like to compare other contexts! 


